In [60]:
import numpy as np
import pandas as pd
import graphviz


In [61]:

data = pd.read_csv(r"C:\Users\admin\Downloads\PlayTennis.csv")

def entropy(data, label):
    total = len(data)
    return -sum((count / total) * np.log2(count / total)
                for count in data[label].value_counts() if count > 0)

def info_gain(feature, data, label):
    total = len(data)
    return entropy(data, label) - sum(
        (len(sub) / total) * entropy(sub, label) 
        for val, sub in data.groupby(feature)
    )

def best_feature(data, label):
    return max(data.columns.drop(label), key=lambda f: info_gain(f, data, label))

def build_tree(data, label):
    if len(data[label].unique()) == 1:
        return data[label].iloc[0]
    
    if data.empty:
        return "?"

    feature = best_feature(data, label)
    tree = {feature: {}}

    for val, sub_data in data.groupby(feature):
        tree[feature][val] = build_tree(sub_data.drop(feature, axis=1), label)
    
    return tree

def predict(tree, instance):
    if not isinstance(tree, dict):
        return tree
    root = next(iter(tree))
    return predict(tree[root][instance[root]], instance) if instance[root] in tree[root] else None

def evaluate(tree, data, label):
    correct = sum(predict(tree, row) == row[label] for _, row in data.iterrows())
    return correct / len(data) if len(data) > 0 else 0

# Create the decision tree and evaluate it
tree = build_tree(data, 'Play Tennis')
accuracy = evaluate(tree, data, 'Play Tennis')

print("Decision Tree:", tree)



Decision Tree: {'Outlook': {'Overcast': 'Yes', 'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}}, 'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}


In [64]:
def add_nodes_edges(tree, parent='Root'):
    for key, value in tree.items():
        node_id = key if parent == 'Root' else f"{parent}_{key}"
        dot.node(node_id, key)
        if parent:
            dot.edge(parent, node_id)
        if isinstance(value, dict):
            add_nodes_edges(value, node_id)
        else:
            dot.node(f"{node_id}_leaf", value)
            dot.edge(node_id, f"{node_id}_leaf")
dot = graphviz.Digraph()
add_nodes_edges(tree)
dot.render('decision_tree', format='png', cleanup=True)


'decision_tree.png'